# mix-SQP demo with different low-rank matrix approximations

In this example, we illustrate how the QR and singular value decompositions of the likelihood matrix can be used to speed up the SQP algorithm.

## Analysis setup

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [24]:
using Distributions
using LowRankApprox
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/mixSQP.jl");

Next, initialize the sequence of pseudorandom numbers.

In [34]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [35]:
x = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [36]:
sd = autoselectmixsd(x,nv = 20);
L  = normlikmatrix(x,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm a few times to precompile the relevant functions.

In [37]:
out = mixSQP(L,lowrank = "svd",verbose = false);
out = mixSQP(L,lowrank = "qr",verbose = false);
out = mixSQP(L,lowrank = "nothing",verbose = false);

Next, run the mix-SQP solver again with the SVD and QR approximations to the likelihood matrix, and with no approximation. The approximation tolerance is set very low, to `1e-10`.

In [38]:
@time outSVD = mixSQP(L,lowrank = "svd",pqrtol = 1e-10,verbose = false);
@time outQR  = mixSQP(L,lowrank = "qr",pqrtol = 1e-10,verbose = false);
@time out    = mixSQP(L,lowrank = "nothing",verbose = false);

  0.327896 seconds (35.40 k allocations: 441.596 MiB, 11.75% gc time)
  0.296751 seconds (38.60 k allocations: 435.244 MiB, 11.71% gc time)
  0.370803 seconds (35.51 k allocations: 810.023 MiB, 16.94% gc time)


You may see a slight improvement in the computation time with the QR and SVD approximations. And the solutions using the low-rank approximations are still very close to the solution without an approximation:

In [39]:
mixobjective(L,outSVD["x"]) - mixobjective(L,out["x"]), 
mixobjective(L,outQR["x"]) - mixobjective(L,out["x"])

(-4.5292836148291826e-10, 1.3096723705530167e-9)

## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [40]:
x = normtmixdatasim(round(Int,1e6));

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 100$ normal densities.

In [41]:
sd = autoselectmixsd(x,nv = 100);
L  = normlikmatrix(x,sd = sd);
size(L)

(1000000, 100)

## Fit mixture model using SQP algorithm 

As before, let's run the mix-SQP solver with the SVD and QR approximations, and with no approximation.

In [42]:
@time outSVD = mixSQP(L,lowrank = "svd",pqrtol = 1e-10,verbose = false);
@time outQR  = mixSQP(L,lowrank = "qr",pqrtol = 1e-10,verbose = false);
@time out    = mixSQP(L,lowrank = "nothing",verbose = false);

 11.614170 seconds (117.49 k allocations: 10.763 GiB, 21.46% gc time)
 12.286642 seconds (126.76 k allocations: 11.294 GiB, 17.92% gc time)
 69.302318 seconds (113.08 k allocations: 70.595 GiB, 9.56% gc time)


In the larger data set, the QR and SVD approximations yield much larger improvements in computation time, which is not surprising because the matrix-vector operations (particularly in computing the gradient and Hessian) dominate the overall computation time.

As before, the solutions using the low-rank approximations are still close to the solution that is obtained without any approximation:

In [43]:
mixobjective(L,outSVD["x"]) - mixobjective(L,out["x"]), 
mixobjective(L,outQR["x"]) - mixobjective(L,out["x"])

(6.834295345470309e-5, 5.512946518138051e-5)

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here.

In [44]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");
versioninfo()

 - Distributions                 0.15.0
 - LowRankApprox                 0.1.1
Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
